In [1]:
from ltr.clickmodels.session import build

doc_costs = {
    0: 0.4,
    1: 0.5,
    2: 0.5,
    3: 0.4,
    4: 0.7,
    5: 0.5,
    6: 0.5,
    7: 0.4,
    8: 0.7,
    9: 0.5,
}

# Sessions for search queries "rambo" and "star trek"
# First bool click, second click is a conversion (empty implies no conversion)
#
# Here 1 is the 'good guy' that always gets a conversion
#      3 gets clicks, but never gets a conversion
sessions = build([
      ('rambo',     ((1, True, True), (2, False), (3, False), (0, False))),
      ('rambo',     ((1, False), (2, True, True), (3, True), (0, False))),
      ('rambo',     ((4, False), (2, False), (1, True, True), (0, True))),
      ('rambo',     ((7, False), (4, False), (1, False), (3, False))),
      ('rambo',     ((1, False), (8, False), (2, False), (3, False))),
      ('rambo',     ((3, True), (2, False), (1, False), (0, False))),
      ('rambo',     ((3, True), (2, False), (1, False), (0, False))),
      ('rambo',     ((9, False), (3, True), (1, True,True), (0, True))),
      ('rambo',     ((6, False), (4, True, True), (1, False), (3, False))),
      ('rambo',     ((0, True,True), (2, True), (1, True,True), (4, True))),


    ])

In [2]:
from ltr.clickmodels.pbm import position_based_model
model = position_based_model(sessions, rounds=50)
model.attracts

defaultdict(<function ltr.clickmodels.pbm.Model.__init__.<locals>.<lambda>()>,
            {('rambo', 2): 0.30524892749163485,
             ('rambo', 3): 0.9997889680727174,
             ('rambo', 0): 0.9999838627093206,
             ('rambo', 1): 0.5081046628637683,
             ('rambo', 4): 0.500867466656602,
             ('rambo', 7): 4.1166079834203487e-75,
             ('rambo', 8): 4.200081298115326e-84,
             ('rambo', 9): 4.1166079834203487e-75,
             ('rambo', 6): 4.1166079834203487e-75})

In [3]:
model.ranks

[0.9976284686244448,
 0.9992604598191843,
 0.6440993392340741,
 0.3000299014351479]

In [4]:
from ltr.clickmodels.conversion import conv_aug_attracts
satisfacts = conv_aug_attracts(model.attracts, sessions, doc_costs)
satisfacts

Counter({('rambo', 1): 0.35567326400463783,
         ('rambo', 2): 0.17442795856664847,
         ('rambo', 3): 0.39991558722908704,
         ('rambo', 0): 0.48570644760167003,
         ('rambo', 4): 0.3881722866588665,
         ('rambo', 7): 1.6466431933681395e-75,
         ('rambo', 8): 2.940056908680728e-84,
         ('rambo', 9): 2.0583039917101744e-75,
         ('rambo', 6): 2.0583039917101744e-75})

In [5]:
def by_query(judgment_dict):
    from collections import defaultdict
    by_query_ranked = defaultdict(list)
    
    for (query, doc), score in judgment_dict.items():
        by_query_ranked[query].append((doc,score))
        
    for query, items in by_query_ranked.items():
        items.sort(key=lambda score_doc: score_doc[1], reverse=True)
        
    return by_query_ranked

def just_doc_ids(docs):
    return [doc[0] for doc in docs]


s_satisfacts = by_query(satisfacts)
s_attracts = by_query(model.attracts)

s_satisfacts

defaultdict(list,
            {'rambo': [(0, 0.48570644760167003),
              (3, 0.39991558722908704),
              (4, 0.3881722866588665),
              (1, 0.35567326400463783),
              (2, 0.17442795856664847),
              (9, 2.0583039917101744e-75),
              (6, 2.0583039917101744e-75),
              (7, 1.6466431933681395e-75),
              (8, 2.940056908680728e-84)]})

In [6]:
s_attracts

defaultdict(list,
            {'rambo': [(0, 0.9999838627093206),
              (3, 0.9997889680727174),
              (1, 0.5081046628637683),
              (4, 0.500867466656602),
              (2, 0.30524892749163485),
              (7, 4.1166079834203487e-75),
              (9, 4.1166079834203487e-75),
              (6, 4.1166079834203487e-75),
              (8, 4.200081298115326e-84)]})

In [7]:
from ltr.helpers.tau import tau, avg_tau

tau(just_doc_ids(s_satisfacts['rambo']),just_doc_ids(s_attracts['rambo']))

0.6666666666666666